In [1]:
# Library imports
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# Source import
wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(wiki_link).text
soup = BeautifulSoup(page, 'html5lib')

# Table definition
table = soup.find('table', class_='wikitable')
row = []

# Fill the table
for tr in table.find_all('tr'):
    td = tr.find_all('td')
    row.append([tr.text.strip() for tr in td if tr.text.strip()])
df = pd.DataFrame(data=row, columns=['PostalCode', 'Borough', 'Neighborhood'])
df = df.drop([0])
df = df[df.Borough != 'Not assigned']
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df.Borough

# Dataset of the postal codes
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [2]:
# Add geaodata into the data
geo_data = pd.read_csv('http://cocl.us/Geospatial_data')
geo_data.head(10)
geo_data.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
geo_data.head(10)
df = pd.merge(df, geo_data, how='left', on='PostalCode')

#Show the content
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [3]:
df.shape

(103, 5)

In [4]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [42]:
# define the world map
toronto_map = folium.Map()

latitude = 43.71 # Y
longitude = -79.5 # X

# define the world map centered around Canada with a higher zoom level
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=9.5)

# display world map
toronto_map

In [43]:
# instantiate a feature group for the incidents in the dataframe
marks = folium.map.FeatureGroup()

# loop and add each to the imarks feature group
for lat, lng, in zip(df.Latitude, df.Longitude):
    marks.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
toronto_map.add_child(marks)